In [ ]:
# 1. Import thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

In [ ]:
# 2. Đọc file csv và gắng index với giá Close
df = pd.read_csv('XRP-USD.csv')
df1=df.reset_index()['Close']

In [ ]:
# 3. Scaler data
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [ ]:
# 4. Chia train test
train_size = int(0.5 * len(df1))
test_size = int(0.3 * len(df1))
val_size = len(df1) - train_size - test_size

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]
val_data = df1[train_size+test_size:]

In [ ]:
# 5. Hàm Create Dataset
# Chuyển từ mảng 1 chiều thành matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
#6. Reshape thành X=t,t+1,t+2..t+99  Y=t+100

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_val, y_val = create_dataset(val_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [ ]:
# 7. Reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

In [ ]:
# 8. Define SImpleRNN Model
model = Sequential()
model.add(SimpleRNN(50, input_shape=(time_step, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
# 9. Fit mô hình với dữ liệu train
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
13/13 [==============================] - 1s 30ms/step - loss: 0.0032 - val_loss: 0.0527
Epoch 2/100
13/13 [==============================] - 0s 17ms/step - loss: 7.5104e-04 - val_loss: 0.0266
Epoch 3/100
13/13 [==============================] - 0s 16ms/step - loss: 5.9376e-04 - val_loss: 0.0282
Epoch 4/100
13/13 [==============================] - 0s 16ms/step - loss: 5.0211e-04 - val_loss: 0.0308
Epoch 5/100
13/13 [==============================] - 0s 17ms/step - loss: 4.6274e-04 - val_loss: 0.0230
Epoch 6/100
13/13 [==============================] - 0s 17ms/step - loss: 4.2808e-04 - val_loss: 0.0223
Epoch 7/100
13/13 [==============================] - 0s 16ms/step - loss: 3.9654e-04 - val_loss: 0.0209
Epoch 8/100
13/13 [==============================] - 0s 15ms/step - loss: 3.6748e-04 - val_loss: 0.0186
Epoch 9/100
13/13 [==============================] - 0s 15ms/step - loss: 3.4791e-04 - val_loss: 0.0177
Epoch 10/100
13/13 [==============================] - 0s 16ms/step -

In [ ]:
# Dự đoán tập test_data dựa trên tập train_data
train_predict = model.predict(X_train)
y_pred = model.predict(X_test)

14/14 [==============================] - 0s 4ms/step


In [ ]:
# Dự đoán tập test_data dựa trên tập train_data
y_pred_val = model.predict(X_val)

9/9 [==============================] - 0s 3ms/step


In [ ]:
# 11. Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)
y_pred_val=scaler.inverse_transform(y_pred_val)

In [ ]:
# 10. Đánh giá độ chính xác thuật toán 6:2:2
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
test_mae = mean_absolute_error(y_pred, test_data[-264:])
test_mape = mean_absolute_percentage_error(y_pred, test_data[-264:])
test_rmse = np.sqrt(np.mean((y_pred - test_data[-264:])**2))
print('Testing MAE:', test_mae)
print('Testing RMSE:', test_rmse)
print('Testing MAPE:', test_mape)

val_mae = mean_absolute_error(y_pred_val, val_data[-265:])
val_rmse = np.sqrt(np.mean((y_pred_val - val_data[-265:])**2))
val_mape = mean_absolute_percentage_error(y_pred_val, val_data[-265:])
print('Validation MAE:', val_mae)
print('Validation RMSE:', val_rmse)
print('Validation MAPE:', val_mape)

In [ ]:
# 10. Đánh giá độ chính xác thuật toán 7:2:1
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
test_mae = mean_absolute_error(y_pred, test_data[-264:])
test_mape = mean_absolute_percentage_error(y_pred, test_data[-264:])
test_rmse = np.sqrt(np.mean((y_pred - test_data[-264:])**2))
print('Testing MAE:', test_mae)
print('Testing RMSE:', test_rmse)
print('Testing MAPE:', test_mape)

val_mae = mean_absolute_error(y_pred_val, val_data[-83:])
val_rmse = np.sqrt(np.mean((y_pred_val[-83:] - val_data[-83:])**2))
val_mape = mean_absolute_percentage_error(y_pred_val, val_data[-83:])
print('Validation MAE:', val_mae)
print('Validation RMSE:', val_rmse)
print('Validation MAPE:', val_mape)

In [ ]:
# 10. Đánh giá độ chính xác thuật toán 5:3:2
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
test_mae = mean_absolute_error(y_pred, test_data[-447:])
test_mape = mean_absolute_percentage_error(y_pred, test_data[-447:])
test_rmse = np.sqrt(np.mean((y_pred - test_data[-447:])**2))
print('Testing MAE:', test_mae)
print('Testing RMSE:', test_rmse)
print('Testing MAPE:', test_mape)

val_mae = mean_absolute_error(y_pred_val, val_data[-265:])
val_rmse = np.sqrt(np.mean((y_pred_val - val_data[-265:])**2))
val_mape = mean_absolute_percentage_error(y_pred_val, val_data[-265:])
print('Validation MAE:', val_mae)
print('Validation RMSE:', val_rmse)
print('Validation MAPE:', val_mape)

Testing MAE: 0.3987489139218139
Testing RMSE: 0.412268172693921
Testing MAPE: 0.4926724785342145
Validation MAE: 0.2505766510785948
Validation RMSE: 0.2520370875382552
Validation MAPE: 0.5969697423982285


In [ ]:
# 13. Dự báo 30 ngày tiếp theo
x_input=val_data[60:].reshape(1,-1)
x_input.shape
temp_input = list(x_input)
temp_input = temp_input[0].tolist()

lst_output = []
n_steps = 100
i = 0
while i < 30:
    if len(temp_input) > 100:
        x_input = np.array(temp_input[-100:])  # Lấy 100 phần tử cuối cùng
        print("{} ngày đầu vào {}".format(i, x_input))
        x_input = x_input.reshape(1, -1)
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = model.predict(x_input, verbose=0)
        print("{} ngày đầu ra {}".format(i, yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
        i += 1
    else:
        x_input = x_input.reshape((1, n_steps, 1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i += 1


In [ ]:
# 14.Vẽ hình

train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))

test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))

test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred))

val_data_index = pd.RangeIndex(start=train_size+test_size, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,scaler.inverse_transform(val_data))

val_data_index = pd.RangeIndex(start=train_size+test_size+101, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,y_pred_val)

prediect_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))

plt.legend(['Train','Test','Predicted Test','Validate','Predicted Validate','Predict30days'])
plt.show()